In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Add the project's files to the python path
# file_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))  # for .py script
file_path = os.path.dirname(os.path.abspath(''))  # for .ipynb notebook
sys.path.append(file_path)

import hydra
from src.utils import init_config, compute_panoptic_metrics, \
    compute_panoptic_metrics_s3dis_6fold, grid_search_panoptic_partition, \
    oracle_superpoint_clustering
import torch
from src.transforms import *
from src.utils.widgets import *
from src.data import *

# Very ugly fix to ignore lightning's warning messages about the
# trainer and modules not being connected
import warnings
warnings.filterwarnings("ignore")

## Select your device, experiment, split, and pretrained model

In [2]:
device_widget = make_device_widget()
task_widget, expe_widget = make_experiment_widgets()
split_widget = make_split_widget()
ckpt_widget = make_checkpoint_file_search_widget()

ToggleButtons(description='👉 Choose a device:', options=(device(type='cpu'), device(type='cuda', index=0), dev…

ToggleButtons(description='👉 Choose a segmentation task:', options=('semantic', 'panoptic'), value='semantic')

ToggleButtons(description='👉 Choose an experiment:', options=('dales', 'dales_11g', 'dales_nano', 'kitti360', …

ToggleButtons(description='👉 Choose a data split:', index=1, options=('train', 'val', 'test'), value='val')

FileChooser(path='/home/binahlab/AI-Labs/superpoint_transformer/notebooks', filename='', title='', show_hidden…

In [3]:
# Summarizing selected task, experiment, split, and checkpoint
print(f"You chose:")
print(f"  - device={device_widget.value}")
print(f"  - task={task_widget.value}")
print(f"  - split={split_widget.value}")
print(f"  - experiment={expe_widget.value}")
print(f"  - ckpt={ckpt_widget.value}")

You chose:
  - device=cuda:0
  - task=panoptic
  - split=train
  - experiment=kitti360_11g
  - ckpt=/home/binahlab/AI-Labs/superpoint_transformer/ckpt/supercluster_kitti360.ckpt


## Parsing the config files
Hydra and OmegaConf are used to parse the `yaml` config files.

❗Make sure you selected a **ckpt file relevant to your experiment** in the previous section. 
You can use our pretrained models for this, or your own checkpoints if you have already trained a model.

In [4]:
# Parse the configs using hydra
cfg = init_config(overrides=[
    f"experiment={task_widget.value}/{expe_widget.value}",
    f"ckpt_path={ckpt_widget.value}"
])

## Datamodule and model instantiation

In [5]:
# Instantiate the datamodule
datamodule = hydra.utils.instantiate(cfg.datamodule)
datamodule.prepare_data()
datamodule.setup()

# Pick among train, val, and test datasets. It is important to note that
# the train dataset produces augmented spherical samples of large 
# scenes, while the val and test dataset load entire tiles at once
if split_widget.value == 'train':
    dataset = datamodule.train_dataset
elif split_widget.value == 'val':
    dataset = datamodule.val_dataset
elif split_widget.value == 'test':
    dataset = datamodule.test_dataset
else:
    raise ValueError(f"Unknown split '{split_widget.value}'")

# Print a summary of the datasets' classes
dataset.print_classes()

# Instantiate the model
model = hydra.utils.instantiate(cfg.model)

# Load pretrained weights from a checkpoint file
if ckpt_widget.value is not None:
    model = model._load_from_checkpoint(cfg.ckpt_path)

# Move model to selected device
model = model.eval().to(device_widget.value)

/home/binahlab/AI-Labs/clever-data/electrical-elements/data/
0   road                 stuff
1   sidewalk             stuff
2   building             thing
3   wall                 stuff
4   fence                stuff
5   pole                 stuff
6   traffic light        stuff
7   traffic sign         stuff
8   vegetation           stuff
9   terrain              stuff
10  person               stuff
11  car                  thing
12  truck                stuff
13  motorcycle           stuff
14  bicycle              stuff
15  ignored              void


## Oracles on a tile sample
We design oracles for estimating the maximum achievable performance of our superpoint-graph-clustering approach on a point cloud. Here, it is important to note that these metrics are computed on a tile but not on the entire dataset. The oracles are computed on a given superpoint partition level. Based on the quality of the partition, we estimate the following:

- `semantic_segmentation_oracle`: assign to each superpoint the most frequent label among the points it contains
- `panoptic_segmentation_oracle`: same as for semantic segmentation + assign each superpoint to the target instance it overlaps the most
- `oracle_superpoint_clustering`: same as for semantic segmentation + assign to each edge the target affinity + compute the graph clustering to form instance predictions

Of course, these oracles are affected by how the superpoint partition has been computed. Besides, the latter is also affected by the graph clustering parameters.

In [6]:
# Get the panoptic annotations for a tile from the dataset 
obj = dataset[0][1].obj

In [7]:
# Compute the semantic segmentation oracle
obj.semantic_segmentation_oracle(dataset.num_classes)

{'oa': tensor(99.1097),
 'macc': tensor(98.0919),
 'miou': tensor(96.2796),
 'iou_per_class': tensor([9.9268e+01, 9.1135e+01, 9.9191e+01, 1.0000e-06, 1.0000e-06, 9.3546e+01,
         1.0000e-06, 9.3659e+01, 9.8860e+01, 9.4754e+01, 1.0000e-06, 9.9825e+01,
         1.0000e-06, 1.0000e-06, 1.0000e-06]),
 'seen_class': tensor([ True,  True,  True, False, False,  True, False,  True,  True,  True,
         False,  True, False, False, False])}

In [8]:
# Compute the panoptic segmentation oracle without graph clustering
obj.panoptic_segmentation_oracle(dataset.num_classes, stuff_classes=dataset.stuff_classes)

{'pq': tensor(0.9623),
 'sq': tensor(0.9623),
 'rq': tensor(1.),
 'pq_modified': tensor(0.9623),
 'pq_thing': tensor(0.9932),
 'sq_thing': tensor(0.9932),
 'rq_thing': tensor(1.),
 'pq_stuff': tensor(0.9520),
 'sq_stuff': tensor(0.9520),
 'rq_stuff': tensor(1.),
 'pq_per_class': tensor([0.9927, 0.9113, 0.9909,    nan,    nan, 0.9355,    nan, 0.9366, 0.9886,
         0.9475,    nan, 0.9954,    nan,    nan,    nan]),
 'sq_per_class': tensor([0.9927, 0.9113, 0.9909,    nan,    nan, 0.9355,    nan, 0.9366, 0.9886,
         0.9475,    nan, 0.9954,    nan,    nan,    nan]),
 'rq_per_class': tensor([1., 1., 1., nan, nan, 1., nan, 1., 1., 1., nan, 1., nan, nan, nan]),
 'precision_per_class': tensor([1., 1., 1., nan, nan, 1., nan, 1., 1., 1., nan, 1., nan, nan, nan]),
 'recall_per_class': tensor([1., 1., 1., nan, nan, 1., nan, 1., 1., 1., nan, 1., nan, nan, nan]),
 'tp_per_class': tensor([1, 1, 3, 0, 0, 1, 0, 1, 1, 1, 0, 9, 0, 0, 0]),
 'fp_per_class': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [9]:
# Compute the panoptic segmentation oracle with graph clustering
oracle_superpoint_clustering(
    dataset[0],
    dataset.num_classes,
    dataset.stuff_classes,
    mode='pas',
    graph_kwargs=dict(
        radius=0.1),
    partition_kwargs=dict(
        regularization=0.1,
        x_weight=1e-3,
        cutoff=300))

{'pq': tensor(0.6792),
 'sq': tensor(0.8376),
 'rq': tensor(0.7759),
 'pq_modified': tensor(0.6792),
 'pq_thing': tensor(0.0862),
 'sq_thing': tensor(0.7199),
 'rq_thing': tensor(0.1036),
 'pq_stuff': tensor(0.8769),
 'sq_stuff': tensor(0.8769),
 'rq_stuff': tensor(1.),
 'pq_per_class': tensor([0.9799, 0.7806, 0.0181,    nan,    nan, 0.7657,    nan, 0.8563, 0.9837,
         0.8952,    nan, 0.1542,    nan,    nan,    nan]),
 'sq_per_class': tensor([0.9799, 0.7806, 0.5574,    nan,    nan, 0.7657,    nan, 0.8563, 0.9837,
         0.8952,    nan, 0.8824,    nan,    nan,    nan]),
 'rq_per_class': tensor([1.0000, 1.0000, 0.0325,    nan,    nan, 1.0000,    nan, 1.0000, 1.0000,
         1.0000,    nan, 0.1748,    nan,    nan,    nan]),
 'precision_per_class': tensor([1.0000, 1.0000, 0.0167,    nan,    nan, 1.0000,    nan, 1.0000, 1.0000,
         1.0000,    nan, 0.0957,    nan,    nan,    nan]),
 'recall_per_class': tensor([1.0000, 1.0000, 0.6667,    nan,    nan, 1.0000,    nan, 1.0000, 1.000

## Grid-searching partition parameters on a tile sample
Our SuperCluster model is trained to predict the input for a graph clustering problem whose solution is a panoptic segmentation of the scene.
Interestingly, with our formulation, the model is **only supervised with local node-wise and edge-wise objectives, without ever needing to compute an actual panoptic partition of the scene during training**.

At inference time, however, we need to decide on some parameters for our graph clustering algorithm.
To this end, a simple post-training grid-search can be used.

We find that similar parameters maximize panoptic segmentation results on all our datasets. 
Here you, we provide utilities for helping you grid-search parameters yourself. See `grid_search_panoptic_partition` docstring for more details on how to use this tool.

In [10]:
# Grid search graph clustering parameters
output, partitions, results = grid_search_panoptic_partition(
    model,
    datamodule.val_dataset,
    i_cloud=0,
    graph_kwargs=dict(
        radius=0.1),
    partition_kwargs=dict(
        regularization=[2e1, 1e1, 5],
        x_weight=[5e-2, 1e-2, 1e-3, 1e-4],
        cutoff=300),
    mode='pas')

100%|██████████| 12/12 [00:02<00:00,  5.80it/s]

    regul.    x_wei.  cutoff     PQ     SQ     RQ
0     20.0  5.00e-02     300  38.76  52.11  51.90
1     20.0  1.00e-02     300  32.79  46.51  41.91
2     20.0  1.00e-03     300  32.81  46.57  41.91
3     20.0  1.00e-04     300  32.78  46.65  41.91
4     10.0  5.00e-02     300  38.90  52.68  51.86
5     10.0  1.00e-02     300  39.16  52.64  51.91
6     10.0  1.00e-03     300  32.77  46.36  41.91
7     10.0  1.00e-04     300  32.77  46.47  41.91
8      5.0  5.00e-02     300  32.18  45.65  41.85
9      5.0  1.00e-02     300  32.58  45.93  41.90
10     5.0  1.00e-03     300  32.61  46.18  41.91
11     5.0  1.00e-04     300  32.85  46.56  41.91


Best panoptic setup: PQ=39.16
   regul.  x_wei.  cutoff
0    10.0    0.01     300

                  PQ     SQ      RQ   PREC.   REC.   TP      FP    FN
road           90.15  90.15  100.00  100.00  100.0  1.0     0.0   0.0
sidewalk       76.64  76.64  100.00  100.00  100.0  1.0     0.0   0.0
building        0.62  66.09    0.94    0.47   37.5  6.0

## Running evaluation on a whole dataset
The above grid search only computes the panoptic segmentation metrics on a single point cloud.
In this section, we provide tools for computing the panoptic metrics on a whole dataset. 

In [11]:
panoptic, instance, semantic = compute_panoptic_metrics(
    model,
    datamodule,
    stage='val',
    graph_kwargs=dict(
        radius=0.1),
    partition_kwargs=dict(
        regularization=1e1,
        x_weight=5e-2,
        cutoff=300))

  0%|          | 0/244 [00:02<?, ?it/s]


RuntimeError: PanopticSegmentationModule is not attached to a `Trainer`.

### S3DIS 6-fold metrics
For S3DIS 6-fold metrics, we provide the following utility for computing metrics.

In [ ]:
fold_ckpt = {
    1: "/path/to/your/s3dis/checkpoint/fold_1.ckpt",
    2: "/path/to/your/s3dis/checkpoint/fold_2.ckpt",
    3: "/path/to/your/s3dis/checkpoint/fold_3.ckpt",
    4: "/path/to/your/s3dis/checkpoint/fold_4.ckpt",
    5: "/path/to/your/s3dis/checkpoint/fold_5.ckpt",
    6: "/path/to/your/s3dis/checkpoint/fold_6.ckpt",
}

experiment_config = f"experiment={task_widget.value}/{expe_widget.value}"

In [ ]:
_ = compute_panoptic_metrics_s3dis_6fold(
    fold_ckpt,
    experiment_config,
    stage='val', 
    graph_kwargs=dict(
        radius=0.1),
    partition_kwargs=dict(
        regularization=10,
        x_weight=1e-3,
        cutoff=300),
    verbose=False)